In [1]:
import numpy as np
import h5py
import pkg_resources
import sys
sys.path.insert(0, '../../')

import ActflowToolbox as actflow

actflow_example_dir = pkg_resources.resource_filename('ActflowToolbox.examples', 'HCP_example_data/')

subjNums = ['100206','108020','117930','126325','133928','143224','153934','164636','174437',
            '183034','194443','204521','212823','268749','322224','385450','463040','529953',
            '587664','656253','731140','814548','877269','978578','100408','108222','118124',
            '126426','134021','144832']
numsubjs=np.shape(subjNums)[0]
numnodes=360
numtimepoints=1195


In [2]:
#Load data
#Preprocessed as described here: https://doi.org/10.1101/560730

#Load resting-state fMRI data; 30 HCP subjects, one run of resting-state fMRI each
restdata=np.zeros((numnodes,numtimepoints,numsubjs))
scount = 0
for subj in subjNums:
    file_path=actflow_example_dir + 'HCP_example_restrun1_subj' + subj + '_data' + '.h5'
    h5f = h5py.File(file_path,'r')
    dataid = 'restdata'
    restdata[:,:,scount] = h5f[dataid][:]
    h5f.close()
    scount += 1

#Load task GLM activations; 30 HCP subjects, 24 task conditions
file_path=actflow_example_dir + 'HCP_example_taskactivations_data' + '.h5'
h5f = h5py.File(file_path,'r')
dataid = 'taskbeta'
activations_bycond = h5f[dataid][:]
h5f.close()


In [3]:
#Run activity flow mapping with Pearson correlation FC
restFC_corr=np.zeros((numnodes,numnodes,numsubjs))
scount=0
for subj in subjNums:
    restFC_corr[:,:,scount]=actflow.connectivity_estimation.corrcoefconn(restdata[:,:,scount])
    scount += 1
print("==Activity flow mapping results, correlation-based resting-state FC, 24 task conditions==")
actflowOutput_restFCCorr_bycond = actflow.actflowcomp.actflowtest(activations_bycond, restFC_corr, print_by_condition=False)


==Activity flow mapping results, correlation-based resting-state FC, 24 task conditions==
===Comparing prediction accuracies between models (similarity between predicted and actual brain activation patterns)===
 
==Condition-wise correlations between predicted and actual activation patterns (calculated for each node separetely):==
--Compare-then-average (calculating prediction accuracies before cross-subject averaging):
Each correlation based on N conditions: 24, p-values based on N subjects (cross-subject variance in correlations): 30
Mean Pearson r=0.62, t-value vs. 0: 42.28, p-value vs. 0: 1.3083678230624174e-27
Mean rank-correlation rho=0.50, t-value vs. 0: 48.81, p-value vs. 0: 2.145198536668705e-29
 
==Condition-wise correlations between predicted and actual activation patterns (calculated for each node separetely):==
--Average-then-compare (calculating prediction accuracies after cross-subject averaging):
Each correlation based on N conditions: 24
Mean Pearson r=0.61
Mean rank-c

In [4]:
%%time
#Run activity flow mapping with ten subjects (to reduce processing time)
restFC_mreg_10subjs=np.zeros((numnodes,numnodes,10))
for scount in np.arange(10):
    restFC_mreg_10subjs[:,:,scount]=actflow.connectivity_estimation.multregconn(restdata[:,:,scount])
print("==Activity flow mapping results, multiple-regression-based resting-state FC, 24 task conditions; 10 subjects==")
actflowOutput_restFCMReg_bycond_10subjs = actflow.actflowcomp.actflowtest(activations_bycond[:,:,0:10], restFC_mreg_10subjs, print_by_condition=False)


==Activity flow mapping results, multiple-regression-based resting-state FC, 24 task conditions; 10 subjects==
===Comparing prediction accuracies between models (similarity between predicted and actual brain activation patterns)===
 
==Condition-wise correlations between predicted and actual activation patterns (calculated for each node separetely):==
--Compare-then-average (calculating prediction accuracies before cross-subject averaging):
Each correlation based on N conditions: 24, p-values based on N subjects (cross-subject variance in correlations): 10
Mean Pearson r=0.81, t-value vs. 0: 43.28, p-value vs. 0: 9.362569560473511e-12
Mean rank-correlation rho=0.67, t-value vs. 0: 46.04, p-value vs. 0: 5.3808266275310385e-12
 
==Condition-wise correlations between predicted and actual activation patterns (calculated for each node separetely):==
--Average-then-compare (calculating prediction accuracies after cross-subject averaging):
Each correlation based on N conditions: 24
Mean Pears

In [5]:
print("===Compare resting-state multregFC actflow predictions to resting-state corrFC actflow prediction, 10 subjects only===")
model_compare_RestMultRegFCVsRestCorrFC_Actflow = actflow.model_compare(target_actvect=actflowOutput_restFCCorr_bycond['actVect_actual_group'][:,:,0:10], model1_actvect=actflowOutput_restFCMReg_bycond_10subjs['actPredVector_bytask_bysubj'], model2_actvect=actflowOutput_restFCCorr_bycond['actPredVector_bytask_bysubj'][:,:,0:10], full_report=True, print_report=True, print_by_condition=False)


===Compare resting-state multregFC actflow predictions to resting-state corrFC actflow prediction, 10 subjects only===
===Comparing prediction accuracies between models (similarity between predicted and actual brain activation patterns)===
 
==Condition-wise correlations between predicted and actual activation patterns (calculated for each node separetely):==
--Compare-then-average (calculating prediction accuracies before cross-subject averaging):
Each correlation based on N conditions: 24, p-values based on N subjects (cross-subject variance in correlations): 10
Model1 mean Pearson r=0.81
Model2 mean Pearson r=0.62
R-value difference = 0.19
Model1 vs. Model2 T-value: 12.46, p-value: 5.572065132326254e-07
Model1 mean rank-correlation rho=0.76
Model2 mean rank-correlation rho=0.57
Rho-value difference = 0.19
Model1 vs. Model2 T-value: 12.70, p-value: 4.7575249396238666e-07
 
==Condition-wise correlations between predicted and actual activation patterns (calculated for each node separet